## Парсинг данных со страницы магистерской программы

In [83]:
import requests
from bs4 import BeautifulSoup
import json

def fetch_program_page(url):
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.text

def parse_program(html):
    soup = BeautifulSoup(html, "html.parser")
    info = {}

    # Название программы
    title_tag = soup.find('h1', class_='Information_information__header__fab3I')
    if title_tag:
        title = title_tag.get_text(strip=True)
        info['program_title'] = title
    

    # Часто задаваемые вопросы (FAQ)
    faqs = []
    faq_header = soup.find("h2", id="faq")
    if faq_header:
        faq_container = faq_header.find_next_sibling("div", class_="Accordion_accordion_container__wkLk9")
        if faq_container:
            faq_items = faq_container.find_all("div", class_="Accordion_accordion__item__A6W5t")
            for item in faq_items:
                # Вопрос
                title_div = item.find("div", class_="Accordion_accordion__title__tSP_0")
                question_tag = title_div.find("h5") if title_div else None
                question = question_tag.get_text(strip=True) if question_tag else None

                # Ответ
                info_div = item.find("div", class_="Accordion_accordion__info__wkCQC")
                answer_div = info_div.find("div") if info_div else None
                answer = answer_div.get_text(" ", strip=True) if answer_div else None

                if question and answer:
                    faqs.append({
                        "question": question,
                        "answer": answer
                    })
    info['faqs'] = faqs

    # Описание программы
    desc = []
    header = soup.find(lambda tag: tag.name in ['h2','h3'] and 'о программе' in tag.get_text(strip=True).lower())
    if header:
        for sib in header.find_next_siblings():
            if sib.name and sib.name.startswith('h'):
                break
            desc.append(sib.get_text(" ", strip=True))
    info['description'] = " ".join(desc)

    career_header = soup.find("h2", id="careers")
    if career_header:
        h5_block = career_header.find_next_sibling("h5")
        if h5_block:
            raw_html = h5_block.decode_contents().replace("<br>", "\n")
            career_text = BeautifulSoup(raw_html, "html.parser").get_text(separator="\n", strip=True)
            info['careers'] = career_text

    admission_header = soup.find("h2", id="admission")
    if admission_header:
        section = admission_header.find_parent("div", class_="Admission_admission__4Rxec")
        if not section:
            return ""

        admission_text = ""
        intro = admission_header.find_next_sibling("h5")
        if intro:
            admission_text += intro.get_text(strip=True) + "\n\n"

        tabpanel = section.find("div", {"role": "tabpanel", "id": "tabs-tabpane-0"})
        if tabpanel:
            accordion = tabpanel.find("div", class_="Accordion_accordion_container__wkLk9")
            if accordion:
                items = accordion.find_all("div", class_="Accordion_accordion__item__A6W5t")
                for item in items:
                    title_div = item.find("div", class_="Accordion_accordion__title__tSP_0")
                    question = title_div.get_text(strip=True) if title_div else ""

                    answer_div = item.find("div", class_="Accordion_accordion__info__wkCQC")
                    answer_main = answer_div.find("div") if answer_div else None
                    answer = answer_main.get_text(" ", strip=True) if answer_main else ""

                    if question and answer:
                        admission_text += f"{question}\n{answer}\n\n"
        info['admission'] = admission_text

    # Основные параметры
    container = soup.find("div", class_="Information_block__K_hTi")
    if not container:
        return params

    cards = container.find_all("div", class_="Information_card__rshys")
    if cards:
        params = {}
        for card in cards:
            header_tag = card.find("div", class_="Information_card__header__6PpVf")
            value_tag = card.find("div", class_="Information_card__text__txwcx")
            if header_tag and value_tag:
                name = header_tag.get_text(strip=True).lower()
                value = value_tag.get_text(strip=True).lower()
                params[name] = value
                
        info['parameters'] = params

    return info

def parse_info(url):
    html = fetch_program_page(url)
    return parse_program(html)


## Парсинг данных из pdf файла с учебным планом программы

In [84]:
import pdfplumber

def parse_disciplines(pdf_path):
    disciplines = []
    with pdfplumber.open(pdf_path) as pdf:
        
        for page in pdf.pages:
            
            bold_texts = set()
            for char in page.chars:
                if any(b in char["fontname"] for b in ["Bold", "bold", "Semibold"]):
                    bold_texts.add(char["text"])
            
            table = page.extract_table()    
    
            for row in table:
                if not row or not row[1]:
                    continue
                text = row[1].strip()
                # Если весь текст строки состоит из жирных символов — вероятно, это заголовок
                if all(ch in bold_texts for ch in text if ch.strip()):
                    continue

                if text == 'Наименование модулей, дисциплин, практики и аттестации':
                    continue
                
                if text:
                    disciplines.append(text)
            
    return disciplines


In [78]:
programs = [
    ("https://abit.itmo.ru/program/master/ai", "10033-abit.pdf"),
    ("https://abit.itmo.ru/program/master/ai_product", "10130-abit.pdf"),
]
program_content = []
for program in programs:
    program_info = parse_info(program[0])
    program_disciplines = parse_disciplines(program[1])

    program_content.append({"info": program_info, "disciplines": program_disciplines})

program_content_ai = json.dumps(program_content[0], ensure_ascii=False, indent=2)
program_content_ai_product = json.dumps(program_content[1], ensure_ascii=False, indent=2)

## Подготовка промпта и отправка в OpenAI API

In [82]:
from openai import OpenAI

base = "https://api.eliza.yandex.net/raw/openai/v1"
key = "{api_key}"

client = OpenAI(api_key=key, base_url=base)

base_prompt = """
Ты — эксперт-консультант ИТМО, специализирующийся на магистратурах по искусственному интеллекту. Ниже представлена информация о двух программах: "Искусственный интеллект" и "Продуктовый искусственный интеллект", включая цели, ключевые дисциплины и учебные планы. Используй эти данные, чтобы дать точный, краткий и полезный ответ на вопрос абитуриента. Отвечай только на вопросы, касающиеся этих программ и обучения на них.

Информация о программе "Искусственный интеллект":
{parsed_content_master_ai}

Информация о программе "Продуктовый искусственный интеллект":
{parsed_content_master_ai_product}

Твоя задача:
1. Дай один точный и обоснованный ответ по содержимому этих образовательных программ.
2. Если вопрос связан со сравнением программ — объясни разницу, какую стоит выбрать и почему.
3. Если дан ввод о бэкграунде или целях — порекомендуй подходящие дисциплины/трек/план обучения.
4. Если вопрос не относится к этим программам — вежливо сообщи, что консультируешь только по этим направлениям.

Формат вывода — лаконичный, содержательный ответ на вопрос в контексте выбора между двумя программами или планирования обучения по одной из них.

Вопрос пользователя:
{user_question}
"""

def get_answer_from_chatgpt(
    parsed_content_master_ai, 
    parsed_content_master_ai_product, 
    user_question
):
    full_prompt = base_prompt.format(
        parsed_content_master_ai=parsed_content_master_ai,
        parsed_content_master_ai_product=parsed_content_master_ai_product,
        user_question=user_question
    )
    
    response = client.chat.completions.create(
        model="gpt-4o-2024-11-20",
        messages=[
            {"role": "system", "content": "Ты консультант по выбору магистратуры ИТМО."},
            {"role": "user", "content": full_prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )

    print(response.choices[0].message.content)

In [81]:
get_answer_from_chatgpt(program_content_ai, program_content_ai_product, "Привет, я закончил ИТМО ПМИ")

Привет! Отлично, что у тебя уже есть базовое образование в ИТМО по прикладной математике и информатике. Оба магистерских направления — "Искусственный интеллект" и "Продуктовый искусственный интеллект" — подойдут для твоего дальнейшего развития.

Если ты хочешь углубиться в техническую часть ИИ, работать с ML-моделями, данными и становиться специалистом уровня Middle (ML Engineer, Data Engineer, Data Analyst), выбирай программу **"Искусственный интеллект"**. Она ориентирована на разработку и внедрение ИИ-решений.

Если же тебя больше интересует управление ИИ-продуктами, навыки продуктового менеджмента, запуск стартапов или развитие ИИ-решений для бизнеса, подойдет программа **"Продуктовый искусственный интеллект"**. Она акцентируется на продуктовой логике и стратегическом управлении.

Выбор зависит от твоих карьерных целей: техническая экспертиза или управление продуктами и командами в области ИИ.
